In [4]:
import os
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt
from sklearn import neural_network as NN
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, cross_val_score

In [5]:
#######################################
symbol_info = np.asarray(['BrentOil', 'Copper', 'CrudeOil', 'Gasoline', 'Gold', 'NaturalGas', 'Platinum', 'Silver',
                          'AUD', 'CNY', 'EUR', 'GBP', 'HKD', 'JPY', 'USD'])

var_info = {'BrentOil': ['Price', 'Open', 'High', 'Low', 'Volume', 'Change'], 
            'Copper' : ['Price', 'Open', 'High', 'Low', 'Volume', 'Change'], 
            'CrudeOil' : ['Price', 'Open', 'High', 'Low', 'Volume', 'Change'], 
            'Gasoline' : ['Price', 'Open', 'High', 'Low', 'Volume', 'Change'], 
            'Gold' : ['Price', 'Open', 'High', 'Low', 'Volume', 'Change'], 
            'NaturalGas' : ['Price', 'Open', 'High', 'Low', 'Volume', 'Change'], 
            'Platinum' : ['Price', 'Open', 'High', 'Low', 'Change'], # Volume에 아무 값도 없음
            'Silver' : ['Price', 'Open', 'High', 'Low', 'Volume', 'Change'], 
            'AUD' : ['Price', 'Open', 'High', 'Low', 'Change'], 
            'CNY' : ['Price', 'Open', 'High', 'Low', 'Change'], 
            'EUR' : ['Price', 'Open', 'High', 'Low', 'Change'], 
            'GBP' : ['Price', 'Open', 'High', 'Low', 'Change'], 
            'HKD' : ['Price', 'Open', 'High', 'Low', 'Change'], 
            'JPY' : ['Price', 'Open', 'High', 'Low', 'Change'], 
            'USD' : ['Price', 'Open', 'High', 'Low', 'Change'] }

col_info = np.asarray(['Price', 'Open', 'High', 'Low', 'Volume', 'Change'])
#######################################
#######################################

In [6]:
def get_data_path(symbol):
    # Return CSV file path given symbol.
    commodity_dir = 'data/commodities'
    currency_dir = 'data/currencies'

    if symbol in ['AUD', 'CNY', 'EUR', 'GBP', 'HKD', 'JPY', 'USD']:
        path = os.path.join(currency_dir, symbol + '.csv')
    else:
        path = os.path.join(commodity_dir, symbol + '.csv')

    return path


def merge_data(start_date, end_date, symbols, p):
    dates = pd.date_range(start_date, end_date, freq='D')
    dates = dates[dates.weekday!=6]
    df = pd.DataFrame(index=dates)

    if 'USD' not in symbols:
        symbols.insert(0, 'USD')

    for symbol in symbols:
        df_temp = pd.read_csv(get_data_path(symbol), index_col="Date", parse_dates=True, na_values=['nan'])
        df_temp = df_temp.reindex(dates)
        df_temp = df_temp[var_info[symbol]]
        df_temp.columns = [symbol + '_' + v for v in var_info[symbol]]  # rename columns
        if symbol in ['BrentOil', 'Copper', 'CrudeOil', 'Gasoline', 'Gold', 'NaturalGas', 'Silver']:
            col = symbol+'_Volume'
            df_temp[col] = pd.to_numeric(df_temp[col].str.slice(0, -1, 1), downcast='float')        
        df = df.join(df_temp)
    df1 = df.fillna(method='ffill', limit=3).fillna(method='bfill', limit=3)
    df2 = df.fillna(method='bfill', limit=3).fillna(method='ffill', limit=3)
    df = df1*p + df2*(1-p)
    
    return df.fillna(0.0)


def make_features(start_date, end_date, symbols, cols, input_days, is_training, p=0.5):
    data = merge_data(start_date, end_date, symbols, p)
    
    if 'Price' not in cols:
        cols.insert(0, 'Price')
    
    # TODO: select columns to use
    columns = list()
    for c in data.columns:
        if c.split("_")[1] in cols:
            columns.append(c)
    data = data[columns]
    USD_price = data['USD_Price']
    x = windowing_x(data, input_days)
    y = windowing_y(USD_price, input_days)

    # split training and test data
    training_x = x[:-10]
    training_y = np.asarray(y[:-10])
    test_x = x[-10]
    test_y = np.asarray(y[-10])
#     print('...... shape of training x: {}'.format(training_x.shape))
#     print('...... shape of test x: {}'.format(test_x.shape))
#     print('...... shape of training y: {}'.format(training_y.shape))
#     print('...... shape of test y: {}\n'.format(test_y.shape))

    return (training_x, training_y) if is_training else (test_x, test_y)


def windowing_y(data, input_days):
    input_size = len(data) - input_days
    windows = [data.iloc[i + input_days : i + input_days + 10] for i in range(input_size)]
    return windows


def windowing_x(data, input_days):
    input_size = len(data) - input_days
    windows_day = np.zeros((input_size,7))
    windows_day[np.arange(input_size), data.index[0:input_size].dayofweek] = 1.0
    data = np.asarray(data)
    windows = np.asarray([np.concatenate([windows_day[i],
                                          np.reshape(data[i:i + input_days],(-1))], axis=0) 
                          for i in range(input_size)])
    return windows


def get_test_dollar_price(start_date, end_date):
    """
    Do not fix this function
    """
    df = pd.read_csv(get_data_path('USD'), index_col="Date", parse_dates=True, na_values=['nan'])
    price = df['Price'].loc[end_date: start_date][:10][::-1]
    return price

In [6]:
def cross_validation(symbol_idx, col_idx, input_days, hidden_layers, l_rate, k, p):
    ### 건드리지 않아도 되는 것 ###
    start_date = '2010-01-01'
    end_date = '2019-04-08'
    symbols = symbol_info[symbol_idx]
    cols = col_info[col_idx]
    model_num = 7 # 앙상블할 모델의 개수
    ################### training and test ###################
    input_hidden_comb = []
    lowest_mae = 100
    best_lr = 0
    best_p = 0
    for p_rate in p:
        for l in l_rate:
            for inputs in input_days:
                for hidden in hidden_layers:
                    training_x, training_y = make_features(start_date, end_date, symbols, cols, inputs, is_training=True)
                    k_fold = KFold(n_splits = k)  #cross validation
                    validation_results = list()
                    f = 0
                    for train_idx, test_idx in k_fold.split(range(training_y.shape[0])):
#                         print('Fold : {}'.format(f))
                        f += 1
                        models = list()
                        for m in range(model_num):
#                             print('...... ensemble {}/{}'.format(m+1,model_num))
                            model_temp = NN.MLPRegressor(hidden_layer_sizes=hidden, activation='relu', 
                                                         solver='adam', alpha=0.0001, batch_size='auto', 
                                                        learning_rate='constant', learning_rate_init=l, 
                                                        power_t=0.5, max_iter=5000, shuffle=True, random_state=m, 
                                                        tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
                                                        nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, 
                                                        beta_1=0.9, beta_2=0.999, epsilon=1e-08)
                            model_temp.fit(training_x[train_idx], training_y[train_idx])
                            models.append(model_temp)
                        predict = np.average([models[idx].predict(training_x[test_idx]) for idx in range(len(models))], axis=0)
                        validation_results.append(mean_absolute_error(predict, training_y[test_idx]))
#                     print('... Cross Validation finished\n')
                    print('\tinput_size: {} / hidden_layers: {} / l_rate: {} / p: {}'.format(inputs,hidden,l,p_rate))
                    print('\tFINAL RESULT')
                    for i, mae in enumerate(validation_results):
                        print('\tFold {} : {}'.format(i, mae))
                    print('\tAVERAGE MAE : ', np.average(validation_results))
                    print('\t'+'#'*50)
                    if np.average(validation_results)<lowest_mae:
                        lowest_mae = np.average(validation_results)
                        input_hidden_comb = [inputs, hidden]
                        best_lr = l
                        best_p = p_rate
    return input_hidden_comb, lowest_mae, best_lr, best_p
    

if __name__ == '__main__':
    ############# HYPER-PARAM #############
    symbol_range = [0,1,2,5,7,9,12,13] # 사용할 변수 설정
    ''' 0: BrentOil, 1: Copper, 2: CrudeOil, 3: Gasoline, 4: Gold, 5: NaturalGas, 6: Platinum, 7: Silver,
        8: AUD, 9: CNY, 10: EUR, 11: GBP, 12: HKD, 13: JPY, 14: USD '''
    col_idx = [0]
    ''' 0: Price, 1: Open, 2: High, 3: Low, 4: Volume, 5: Change'''
    p = [0.0, 0.1, 0.3]
    input_days = [3,4,5,6]
    hidden_layers = [(100,50,25),(32,32),(32,32,32)] 
    l_rate = [0.01]
    k = 5 
    
    
    best_dict = {}
    print('%'*100)
    for i in range(2**len(symbol_range)):
        symbol_idx = [14]
        index = list()
        j = i
        for s in range(len(symbol_range)):
            if j%2 == 1: 
                symbol_idx += [symbol_range[s]]
                index += ['O']
            else: index += ['X']
            j = j//2
        
        
        print(symbol_info[symbol_idx], col_info[col_idx])
        input_hidden_comb, lowest_mae, best_lr, best_p = cross_validation(symbol_idx, col_idx, input_days, hidden_layers, l_rate, k, p)
        best_list = [input_hidden_comb, lowest_mae, best_lr, best_p]

        best_dict[tuple(index)] = best_list
        print('BEST :', input_hidden_comb, lowest_mae, best_lr, best_p)
        print('%'*100)
    

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
['USD'] ['Price']
	input_size: 3 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 12.645098802126961
	Fold 1 : 7.924393975661877
	Fold 2 : 8.035892369279624
	Fold 3 : 11.805417416937496
	Fold 4 : 14.764370714983912
	AVERAGE MAE :  11.035034655797974
	##################################################
	input_size: 3 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 12.909631365004202
	Fold 1 : 8.09712286061707
	Fold 2 : 8.060261763542897
	Fold 3 : 11.98270618352866
	Fold 4 : 14.786943980620233
	AVERAGE MAE :  11.167333230662612
	##################################################
	input_size: 3 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 12.834729938434236
	Fold 1 : 8.003723680221562
	Fold 2 : 7.957147169100684
	Fold 3 : 11.874552611156735
	Fold 4 : 14.694767578013526
	AVERAGE MAE :  11.072984195385349

	input_size: 3 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 12.834729938434236
	Fold 1 : 8.003723680221562
	Fold 2 : 7.957147169100684
	Fold 3 : 11.874552611156735
	Fold 4 : 14.694767578013526
	AVERAGE MAE :  11.072984195385349
	##################################################
	input_size: 4 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.28770620999887
	Fold 1 : 8.572699667675607
	Fold 2 : 8.461865967038104
	Fold 3 : 12.107715378996817
	Fold 4 : 14.891315974709158
	AVERAGE MAE :  11.464260639683712
	##################################################
	input_size: 4 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.249938521535034
	Fold 1 : 8.338991028779256
	Fold 2 : 8.389907818992723
	Fold 3 : 12.33074692713904
	Fold 4 : 14.93754798959721
	AVERAGE MAE :  11.449426457208654
	##################################################
	input_size: 4 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.

	input_size: 4 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.1
	FINAL RESULT
	Fold 0 : 13.428465295183159
	Fold 1 : 8.565113343153222
	Fold 2 : 8.674754909034558
	Fold 3 : 13.72834862036098
	Fold 4 : 14.841915368079441
	AVERAGE MAE :  11.847719507162271
	##################################################
	input_size: 4 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.1
	FINAL RESULT
	Fold 0 : 13.2959638509959
	Fold 1 : 8.34051568690992
	Fold 2 : 8.869097416878654
	Fold 3 : 13.1982977030087
	Fold 4 : 14.858804831698091
	AVERAGE MAE :  11.712535897898253
	##################################################
	input_size: 5 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.1
	FINAL RESULT
	Fold 0 : 13.460270294199304
	Fold 1 : 8.619541182607279
	Fold 2 : 9.058461073366797
	Fold 3 : 14.772476829826937
	Fold 4 : 15.35871775409081
	AVERAGE MAE :  12.253893426818227
	##################################################
	input_size: 5 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.1
	FINAL

	input_size: 5 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 13.337527023166937
	Fold 1 : 8.585799285328015
	Fold 2 : 9.222156203659996
	Fold 3 : 12.5320940423697
	Fold 4 : 14.933531021168466
	AVERAGE MAE :  11.722221515138623
	##################################################
	input_size: 5 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 13.690739374279042
	Fold 1 : 8.516038892516598
	Fold 2 : 8.689224806151072
	Fold 3 : 12.606739921151108
	Fold 4 : 15.121834464128545
	AVERAGE MAE :  11.724915491645273
	##################################################
	input_size: 5 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 13.274033181452566
	Fold 1 : 8.47497717442532
	Fold 2 : 8.402696288994383
	Fold 3 : 12.528297009497328
	Fold 4 : 14.954030193174935
	AVERAGE MAE :  11.526806769508907
	##################################################
	input_size: 6 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0

	input_size: 6 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.803065448869367
	Fold 1 : 8.704097549071815
	Fold 2 : 8.747868282329204
	Fold 3 : 13.199999757368122
	Fold 4 : 15.143079400594658
	AVERAGE MAE :  11.919622087646633
	##################################################
	input_size: 6 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.656663043106459
	Fold 1 : 8.553222241957783
	Fold 2 : 8.499111339111803
	Fold 3 : 12.56574004639818
	Fold 4 : 15.095496674653436
	AVERAGE MAE :  11.674046669045532
	##################################################
	input_size: 6 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.74242474597843
	Fold 1 : 9.077571694622069
	Fold 2 : 9.86334010623244
	Fold 3 : 12.621774198556746
	Fold 4 : 15.065706057433516
	AVERAGE MAE :  12.07416336056464
	##################################################
BEST : [3, (32, 32)] 11.133091046775025 0.01 0.0
%%%%%%%%%%%%%%%%%%%

...

	input_size: 4 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.103891926718873
	Fold 1 : 8.7469440539621
	Fold 2 : 8.470736753140084
	Fold 3 : 13.538982653632937
	Fold 4 : 15.360828990628718
	AVERAGE MAE :  11.844276875616542
	##################################################
	input_size: 5 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.924102206240864
	Fold 1 : 10.178646783077816
	Fold 2 : 9.473346616339905
	Fold 3 : 12.925710411682317
	Fold 4 : 15.274479201838853
	AVERAGE MAE :  12.355257043835952
	##################################################
	input_size: 5 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.65929734511812
	Fold 1 : 9.774727106530039
	Fold 2 : 8.533268478693333
	Fold 3 : 13.184290404387244
	Fold 4 : 15.026677762415565
	AVERAGE MAE :  12.03565221942886
	##################################################
	input_size: 5 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.

	input_size: 5 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.1
	FINAL RESULT
	Fold 0 : 13.570855338653772
	Fold 1 : 10.413811504961572
	Fold 2 : 9.375311103311443
	Fold 3 : 12.817914841483901
	Fold 4 : 15.60952809991785
	AVERAGE MAE :  12.357484177665707
	##################################################
	input_size: 5 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.1
	FINAL RESULT
	Fold 0 : 13.393998461165253
	Fold 1 : 9.72087338907826
	Fold 2 : 8.99015404536618
	Fold 3 : 13.436694232207548
	Fold 4 : 15.542332187409837
	AVERAGE MAE :  12.216810463045416
	##################################################
	input_size: 6 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.1
	FINAL RESULT
	Fold 0 : 14.181313298054913
	Fold 1 : 10.89229371187276
	Fold 2 : 8.649426674152103
	Fold 3 : 14.66654844078774
	Fold 4 : 15.71072162672139
	AVERAGE MAE :  12.820060750317783
	##################################################
	input_size: 6 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.1
	FI

	input_size: 6 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 14.413331851733407
	Fold 1 : 9.263184721821485
	Fold 2 : 9.383975626052353
	Fold 3 : 13.402146004411966
	Fold 4 : 15.468223770065535
	AVERAGE MAE :  12.386172394816949
	##################################################
	input_size: 6 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 14.223033724388122
	Fold 1 : 11.038953605216708
	Fold 2 : 9.272294337155476
	Fold 3 : 14.868177902670206
	Fold 4 : 17.1668144877986
	AVERAGE MAE :  13.313854811445822
	##################################################
	input_size: 6 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 14.215653110882533
	Fold 1 : 11.206063149060714
	Fold 2 : 8.996123304334196
	Fold 3 : 15.177341919731514
	Fold 4 : 15.864180219996005
	AVERAGE MAE :  13.091872340800993
	##################################################
	input_size: 3 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p

	input_size: 3 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 13.413264222609957
	Fold 1 : 8.083850490852146
	Fold 2 : 8.020955933420929
	Fold 3 : 12.72718953404216
	Fold 4 : 14.71262283523186
	AVERAGE MAE :  11.39157660323141
	##################################################
	input_size: 3 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 12.689103484338364
	Fold 1 : 8.178875899781037
	Fold 2 : 8.489873803011768
	Fold 3 : 13.181401982627335
	Fold 4 : 14.816517014705534
	AVERAGE MAE :  11.471154436892807
	##################################################
	input_size: 3 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.0
	FINAL RESULT
	Fold 0 : 12.794260869825154
	Fold 1 : 8.922893040616199
	Fold 2 : 8.990893031428962
	Fold 3 : 13.29684527251963
	Fold 4 : 14.897287037151315
	AVERAGE MAE :  11.780435850308251
	##################################################
	input_size: 4 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.

	input_size: 4 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.167935903306665
	Fold 1 : 8.33911076738967
	Fold 2 : 8.436405510052069
	Fold 3 : 13.737290124468151
	Fold 4 : 15.3076031958566
	AVERAGE MAE :  11.79766910021463
	##################################################
	input_size: 4 / hidden_layers: (32, 32) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.29032338635175
	Fold 1 : 8.703196372026133
	Fold 2 : 8.641022082308577
	Fold 3 : 14.558397703720246
	Fold 4 : 15.269789638352503
	AVERAGE MAE :  12.09254583655184
	##################################################
	input_size: 4 / hidden_layers: (32, 32, 32) / l_rate: 0.01 / p: 0.3
	FINAL RESULT
	Fold 0 : 13.136890186249763
	Fold 1 : 10.084652088369337
	Fold 2 : 8.415510539208826
	Fold 3 : 12.7925833208354
	Fold 4 : 15.260133570570252
	AVERAGE MAE :  11.937953941046716
	##################################################
	input_size: 5 / hidden_layers: (100, 50, 25) / l_rate: 0.01 / p: 0.3
	

In [7]:
best_dict

{('X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'): [[3, (100, 50, 25)],
  11.035034655797974,
  0.01,
  0.0],
 ('O', 'X', 'X', 'X', 'X', 'X', 'X', 'X'): [[3, (100, 50, 25)],
  11.090862428010613,
  0.01,
  0.0],
 ('X', 'O', 'X', 'X', 'X', 'X', 'X', 'X'): [[3, (32, 32)],
  11.133091046775025,
  0.01,
  0.0],
 ('O', 'O', 'X', 'X', 'X', 'X', 'X', 'X'): [[3, (32, 32, 32)],
  11.14988790906571,
  0.01,
  0.0],
 ('X', 'X', 'O', 'X', 'X', 'X', 'X', 'X'): [[3, (32, 32)],
  11.125213536580995,
  0.01,
  0.0],
 ('O', 'X', 'O', 'X', 'X', 'X', 'X', 'X'): [[3, (32, 32, 32)],
  11.265507578333754,
  0.01,
  0.0],
 ('X', 'O', 'O', 'X', 'X', 'X', 'X', 'X'): [[3, (32, 32)],
  11.02818493983728,
  0.01,
  0.0],
 ('O', 'O', 'O', 'X', 'X', 'X', 'X', 'X'): [[3, (100, 50, 25)],
  11.169103033672743,
  0.01,
  0.0],
 ('X', 'X', 'X', 'O', 'X', 'X', 'X', 'X'): [[3, (32, 32)],
  11.104545943821886,
  0.01,
  0.0],
 ('O', 'X', 'X', 'O', 'X', 'X', 'X', 'X'): [[3, (32, 32, 32)],
  11.17229128879615,
  0.01,
  0.0],
 ('X',

In [8]:
b = pd.DataFrame(best_dict)
b.to_pickle('best_dict.pkl')

In [37]:
####### 실제 제출해야하는 코드 #######

def main():
    ################### HYPER-PARAM ###################
#     symbol_idx = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    symbol_idx = [3,12] # 사용할 변수 설정 ##[0,1,3,5,7,9,12,13]
    ''' 0: BrentOil, 1: Copper, 2: CrudeOil, 3: Gasoline, 4: Gold, 5: NaturalGas, 6: Platinum, 7: Silver,
        8: AUD, 9: CNY, 10: EUR, 11: GBP, 12: HKD, 13: JPY, 14: USD '''
    col_idx = [0]
    ''' 0: Price, 1: Open, 2: High, 3: Low, 4: Volume, 5: Change'''
    p = 0
    input_days = 3
    hidden = (100,50,20)
    l_rate = 0.01
    is_training = True # 참일 때는 학습+테스트 모두 수행. 거짓일 때는 테스트만 수행.
    
    ### 건드리지 않아도 되는 것 ###
    start_date = '2010-01-01'
    end_date = '2019-04-08'
    filename = 'teamOO_model.pkl'
    symbols = symbol_info[symbol_idx]
    cols = col_info[col_idx]
    model_num = 7 # 앙상블할 모델의 개수
    

    
    ################### training step ###################
    if is_training:
        print('Training ...')
        training_x, training_y = make_features(start_date, end_date, symbols, cols, input_days, is_training)

        models = list()
        for i in range(model_num):
            print('...... ensemble {}'.format(i+1))
            model_temp = NN.MLPRegressor(hidden_layer_sizes=hidden, activation='relu', 
                                    solver='adam', alpha=0.0001, batch_size='auto', 
                                    learning_rate='constant', learning_rate_init=l_rate, 
                                    power_t=0.5, max_iter=5000, shuffle=True, random_state=i, 
                                    tol=0.0001, verbose=True, warm_start=False, momentum=0.9, 
                                    nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, 
                                    beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            model_temp.fit(training_x, training_y)
            models.append(model_temp)
        pickle.dump(models, open(filename, 'wb'))
        print('...... saved {}'.format(filename))
        print('... Training finished\n\n')
    
    ###################   test step   ###################
    print('Test ...')
    test_x, test_y = make_features(start_date, end_date, is_training=False)

    # inspect test data
    assert test_y.tolist() == get_test_dollar_price(start_date, end_date).tolist(), 'your test data is wrong!'

    loaded_models = pickle.load(open(filename, 'rb'))
    print('...... load complete')
#     print('...... parameter :' + str(loaded_models[0].get_params()))
    predict = np.average([loaded_models[i].predict([test_x]) for i in range(len(loaded_models))], axis=0)
    print('... Test finished\n\n')
    
    print('FINAL RESULT')
    print('MAE : ', mean_absolute_error(np.reshape(predict, -1), test_y))


In [38]:
if __name__ == '__main__':
    main()

Training ...


AttributeError: 'numpy.ndarray' object has no attribute 'insert'